In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, retuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    aloss2 += -tf.reduce_mean(eQs_logits) # max
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:22.0000 R:22.0000 rate:0.0440 aloss:0.6823 eloss:2.7296 aloss2:0.7847 exploreP:0.9978
Episode:1 meanR:21.0000 R:20.0000 rate:0.0400 aloss:0.6893 eloss:2.7431 aloss2:0.7915 exploreP:0.9959
Episode:2 meanR:27.3333 R:40.0000 rate:0.0800 aloss:0.6918 eloss:2.6936 aloss2:0.7770 exploreP:0.9919
Episode:3 meanR:26.0000 R:22.0000 rate:0.0440 aloss:0.6856 eloss:2.6618 aloss2:0.7622 exploreP:0.9898
Episode:4 meanR:22.8000 R:10.0000 rate:0.0200 aloss:0.6908 eloss:2.6248 aloss2:0.7582 exploreP:0.9888
Episode:5 meanR:20.8333 R:11.0000 rate:0.0220 aloss:0.6893 eloss:2.6284 aloss2:0.7625 exploreP:0.9877
Episode:6 meanR:20.7143 R:20.0000 rate:0.0400 aloss:0.6894 eloss:2.6330 aloss2:0.7560 exploreP:0.9857
Episode:7 meanR:19.7500 R:13.0000 rate:0.0260 aloss:0.6988 eloss:2.6433 aloss2:0.7600 exploreP:0.9845
Episode:8 meanR:19.6667 R:19.0000 rate:0.0380 aloss:0.7026 eloss:2.6379 aloss2:0.7421 exploreP:0.9826
Episode:9 meanR:20.5000 R:28.0000 rate:0.0560 aloss:0.6909 eloss:2.6053 aloss2:0.7

Episode:80 meanR:22.7407 R:28.0000 rate:0.0560 aloss:0.6807 eloss:2.0573 aloss2:0.5968 exploreP:0.8335
Episode:81 meanR:23.0000 R:44.0000 rate:0.0880 aloss:0.6825 eloss:2.0581 aloss2:0.5894 exploreP:0.8298
Episode:82 meanR:22.9157 R:16.0000 rate:0.0320 aloss:0.6810 eloss:2.0768 aloss2:0.6107 exploreP:0.8285
Episode:83 meanR:22.8095 R:14.0000 rate:0.0280 aloss:0.6879 eloss:2.0772 aloss2:0.6044 exploreP:0.8274
Episode:84 meanR:23.0000 R:39.0000 rate:0.0780 aloss:0.6871 eloss:2.0957 aloss2:0.6023 exploreP:0.8242
Episode:85 meanR:23.3023 R:49.0000 rate:0.0980 aloss:0.6960 eloss:2.0754 aloss2:0.6081 exploreP:0.8202
Episode:86 meanR:23.2299 R:17.0000 rate:0.0340 aloss:0.6860 eloss:2.0698 aloss2:0.6009 exploreP:0.8188
Episode:87 meanR:23.3750 R:36.0000 rate:0.0720 aloss:0.6823 eloss:2.0592 aloss2:0.6196 exploreP:0.8159
Episode:88 meanR:23.3933 R:25.0000 rate:0.0500 aloss:0.6862 eloss:2.0880 aloss2:0.6286 exploreP:0.8139
Episode:89 meanR:23.2778 R:13.0000 rate:0.0260 aloss:0.6844 eloss:2.1346 

Episode:159 meanR:39.9100 R:25.0000 rate:0.0500 aloss:0.6741 eloss:2.1123 aloss2:1.3075 exploreP:0.5885
Episode:160 meanR:39.9100 R:22.0000 rate:0.0440 aloss:0.6761 eloss:2.1067 aloss2:1.3220 exploreP:0.5873
Episode:161 meanR:40.5500 R:76.0000 rate:0.1520 aloss:0.6805 eloss:2.1046 aloss2:1.3277 exploreP:0.5829
Episode:162 meanR:40.6900 R:26.0000 rate:0.0520 aloss:0.6803 eloss:2.1253 aloss2:1.3390 exploreP:0.5814
Episode:163 meanR:40.7700 R:22.0000 rate:0.0440 aloss:0.6797 eloss:2.1160 aloss2:1.3209 exploreP:0.5802
Episode:164 meanR:41.0900 R:50.0000 rate:0.1000 aloss:0.6788 eloss:2.0917 aloss2:1.3398 exploreP:0.5773
Episode:165 meanR:41.2200 R:24.0000 rate:0.0480 aloss:0.6813 eloss:2.1075 aloss2:1.3629 exploreP:0.5759
Episode:166 meanR:41.4700 R:47.0000 rate:0.0940 aloss:0.6804 eloss:2.0898 aloss2:1.3634 exploreP:0.5733
Episode:167 meanR:42.3300 R:102.0000 rate:0.2040 aloss:0.6800 eloss:2.0938 aloss2:1.3911 exploreP:0.5676
Episode:168 meanR:42.7100 R:70.0000 rate:0.1400 aloss:0.6760 el

Episode:238 meanR:43.4400 R:46.0000 rate:0.0920 aloss:0.6800 eloss:1.9758 aloss2:1.7487 exploreP:0.4272
Episode:239 meanR:43.1200 R:21.0000 rate:0.0420 aloss:0.6709 eloss:1.9943 aloss2:1.7129 exploreP:0.4263
Episode:240 meanR:42.6900 R:12.0000 rate:0.0240 aloss:0.6819 eloss:1.9778 aloss2:1.7605 exploreP:0.4258
Episode:241 meanR:42.0700 R:46.0000 rate:0.0920 aloss:0.6777 eloss:1.9837 aloss2:1.7439 exploreP:0.4239
Episode:242 meanR:42.1200 R:84.0000 rate:0.1680 aloss:0.6773 eloss:2.0290 aloss2:1.7450 exploreP:0.4204
Episode:243 meanR:42.5000 R:77.0000 rate:0.1540 aloss:0.6794 eloss:1.9619 aloss2:1.7548 exploreP:0.4173
Episode:244 meanR:43.2800 R:95.0000 rate:0.1900 aloss:0.6766 eloss:1.9738 aloss2:1.7433 exploreP:0.4134
Episode:245 meanR:43.8100 R:103.0000 rate:0.2060 aloss:0.6799 eloss:1.9671 aloss2:1.7540 exploreP:0.4093
Episode:246 meanR:43.0700 R:13.0000 rate:0.0260 aloss:0.6736 eloss:1.9691 aloss2:1.7624 exploreP:0.4088
Episode:247 meanR:42.9800 R:10.0000 rate:0.0200 aloss:0.6844 el

Episode:317 meanR:37.8800 R:43.0000 rate:0.0860 aloss:0.6845 eloss:1.8842 aloss2:1.8525 exploreP:0.3208
Episode:318 meanR:37.0400 R:14.0000 rate:0.0280 aloss:0.6775 eloss:1.8659 aloss2:1.8769 exploreP:0.3204
Episode:319 meanR:36.5400 R:28.0000 rate:0.0560 aloss:0.6757 eloss:1.8710 aloss2:1.8348 exploreP:0.3195
Episode:320 meanR:36.5400 R:22.0000 rate:0.0440 aloss:0.6846 eloss:1.8721 aloss2:1.8580 exploreP:0.3189
Episode:321 meanR:36.3000 R:75.0000 rate:0.1500 aloss:0.6817 eloss:1.8594 aloss2:1.8524 exploreP:0.3166
Episode:322 meanR:35.7200 R:64.0000 rate:0.1280 aloss:0.6767 eloss:1.8454 aloss2:1.8753 exploreP:0.3146
Episode:323 meanR:35.5100 R:74.0000 rate:0.1480 aloss:0.6822 eloss:1.8588 aloss2:1.8706 exploreP:0.3124
Episode:324 meanR:35.9500 R:59.0000 rate:0.1180 aloss:0.6828 eloss:1.8580 aloss2:1.8708 exploreP:0.3106
Episode:325 meanR:36.2600 R:51.0000 rate:0.1020 aloss:0.6775 eloss:1.8508 aloss2:1.8825 exploreP:0.3090
Episode:326 meanR:35.9100 R:43.0000 rate:0.0860 aloss:0.6844 elo

Episode:396 meanR:70.5400 R:53.0000 rate:0.1060 aloss:0.6801 eloss:1.7250 aloss2:1.9075 exploreP:0.1800
Episode:397 meanR:70.9300 R:54.0000 rate:0.1080 aloss:0.6819 eloss:1.7052 aloss2:1.8810 exploreP:0.1791
Episode:398 meanR:71.6500 R:83.0000 rate:0.1660 aloss:0.6796 eloss:1.7054 aloss2:1.9079 exploreP:0.1777
Episode:399 meanR:73.6000 R:212.0000 rate:0.4240 aloss:0.6768 eloss:1.7127 aloss2:1.8949 exploreP:0.1742
Episode:400 meanR:75.1900 R:172.0000 rate:0.3440 aloss:0.6814 eloss:1.7057 aloss2:1.9203 exploreP:0.1714
Episode:401 meanR:76.8000 R:172.0000 rate:0.3440 aloss:0.6797 eloss:1.7184 aloss2:1.9068 exploreP:0.1686
Episode:402 meanR:78.5300 R:181.0000 rate:0.3620 aloss:0.6760 eloss:1.6713 aloss2:1.9106 exploreP:0.1658
Episode:403 meanR:79.8700 R:182.0000 rate:0.3640 aloss:0.6810 eloss:1.6795 aloss2:1.9153 exploreP:0.1630
Episode:404 meanR:81.1500 R:185.0000 rate:0.3700 aloss:0.6803 eloss:1.6912 aloss2:1.9077 exploreP:0.1602
Episode:405 meanR:82.2400 R:199.0000 rate:0.3980 aloss:0.6

Episode:474 meanR:161.0700 R:213.0000 rate:0.4260 aloss:0.6867 eloss:1.8639 aloss2:1.8186 exploreP:0.0508
Episode:475 meanR:162.6600 R:205.0000 rate:0.4100 aloss:0.6860 eloss:1.8910 aloss2:1.8089 exploreP:0.0500
Episode:476 meanR:164.4300 R:224.0000 rate:0.4480 aloss:0.6715 eloss:1.8921 aloss2:1.7829 exploreP:0.0491
Episode:477 meanR:166.0400 R:210.0000 rate:0.4200 aloss:0.6831 eloss:1.8309 aloss2:1.7913 exploreP:0.0483
Episode:478 meanR:166.6900 R:196.0000 rate:0.3920 aloss:0.6773 eloss:1.8175 aloss2:1.7945 exploreP:0.0475
Episode:479 meanR:168.3800 R:228.0000 rate:0.4560 aloss:0.6795 eloss:1.8552 aloss2:1.7904 exploreP:0.0467
Episode:480 meanR:169.5700 R:186.0000 rate:0.3720 aloss:0.6704 eloss:1.8628 aloss2:1.7711 exploreP:0.0460
Episode:481 meanR:170.8500 R:180.0000 rate:0.3600 aloss:0.6813 eloss:1.8662 aloss2:1.7752 exploreP:0.0454
Episode:482 meanR:172.3600 R:203.0000 rate:0.4060 aloss:0.6834 eloss:1.7927 aloss2:1.7406 exploreP:0.0447
Episode:483 meanR:173.8200 R:204.0000 rate:0.4

Episode:552 meanR:192.3000 R:92.0000 rate:0.1840 aloss:0.6673 eloss:1.8876 aloss2:1.6149 exploreP:0.0192
Episode:553 meanR:191.6300 R:96.0000 rate:0.1920 aloss:0.6690 eloss:1.8670 aloss2:1.6907 exploreP:0.0191
Episode:554 meanR:190.4600 R:97.0000 rate:0.1940 aloss:0.6657 eloss:1.8743 aloss2:1.5417 exploreP:0.0190
Episode:555 meanR:189.1900 R:80.0000 rate:0.1600 aloss:0.6690 eloss:1.8579 aloss2:1.5880 exploreP:0.0189
Episode:556 meanR:187.9000 R:70.0000 rate:0.1400 aloss:0.6669 eloss:1.9448 aloss2:1.6338 exploreP:0.0189
Episode:557 meanR:186.9600 R:88.0000 rate:0.1760 aloss:0.6830 eloss:1.9209 aloss2:1.5539 exploreP:0.0188
Episode:558 meanR:185.8600 R:64.0000 rate:0.1280 aloss:0.6472 eloss:1.8631 aloss2:1.5580 exploreP:0.0187
Episode:559 meanR:184.3200 R:66.0000 rate:0.1320 aloss:0.6691 eloss:1.8620 aloss2:1.5971 exploreP:0.0187
Episode:560 meanR:183.1700 R:78.0000 rate:0.1560 aloss:0.6792 eloss:1.8582 aloss2:1.6271 exploreP:0.0186
Episode:561 meanR:182.0600 R:82.0000 rate:0.1640 aloss:

Episode:631 meanR:74.5700 R:64.0000 rate:0.1280 aloss:0.6599 eloss:1.7892 aloss2:1.6433 exploreP:0.0156
Episode:632 meanR:70.0600 R:49.0000 rate:0.0980 aloss:0.6711 eloss:1.8654 aloss2:1.6728 exploreP:0.0156
Episode:633 meanR:68.5200 R:54.0000 rate:0.1080 aloss:0.6604 eloss:1.7769 aloss2:1.6485 exploreP:0.0156
Episode:634 meanR:67.4100 R:44.0000 rate:0.0880 aloss:0.6724 eloss:1.7295 aloss2:1.7244 exploreP:0.0155
Episode:635 meanR:67.0500 R:56.0000 rate:0.1120 aloss:0.6912 eloss:1.7476 aloss2:1.6078 exploreP:0.0155
Episode:636 meanR:66.1700 R:48.0000 rate:0.0960 aloss:0.6774 eloss:1.7339 aloss2:1.6409 exploreP:0.0155
Episode:637 meanR:65.8100 R:68.0000 rate:0.1360 aloss:0.6748 eloss:1.6950 aloss2:1.6754 exploreP:0.0154
Episode:638 meanR:65.4800 R:55.0000 rate:0.1100 aloss:0.6725 eloss:1.7526 aloss2:1.6180 exploreP:0.0154
Episode:639 meanR:65.2400 R:57.0000 rate:0.1140 aloss:0.6560 eloss:1.7700 aloss2:1.5866 exploreP:0.0154
Episode:640 meanR:64.9500 R:70.0000 rate:0.1400 aloss:0.6731 elo

Episode:710 meanR:161.6400 R:82.0000 rate:0.1640 aloss:0.6439 eloss:2.2039 aloss2:1.2193 exploreP:0.0113
Episode:711 meanR:161.7700 R:77.0000 rate:0.1540 aloss:0.5619 eloss:2.3244 aloss2:1.2622 exploreP:0.0113
Episode:712 meanR:162.0700 R:90.0000 rate:0.1800 aloss:0.6353 eloss:2.1467 aloss2:1.2810 exploreP:0.0112
Episode:713 meanR:162.3900 R:98.0000 rate:0.1960 aloss:0.6236 eloss:2.1246 aloss2:1.2786 exploreP:0.0112
Episode:714 meanR:162.7900 R:100.0000 rate:0.2000 aloss:0.6168 eloss:2.2095 aloss2:1.2884 exploreP:0.0112
Episode:715 meanR:162.9500 R:74.0000 rate:0.1480 aloss:0.5955 eloss:2.2329 aloss2:1.2838 exploreP:0.0112
Episode:716 meanR:163.6700 R:126.0000 rate:0.2520 aloss:0.6003 eloss:2.1541 aloss2:1.2600 exploreP:0.0112
Episode:717 meanR:165.1500 R:201.0000 rate:0.4020 aloss:0.6146 eloss:2.2030 aloss2:1.2527 exploreP:0.0112
Episode:718 meanR:166.2400 R:164.0000 rate:0.3280 aloss:0.6015 eloss:2.1875 aloss2:1.2704 exploreP:0.0112
Episode:719 meanR:166.6100 R:84.0000 rate:0.1680 al

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
